In [ ]:
import asyncio
import os
import pandas as pd
from glob import glob
from dotenv import load_dotenv
import sys
import env_defs as ed
import utils

In [ ]:
import traceback as tb

### Catalogue the Files

In [ ]:
logistics_files = glob(f'{ed.raw_files}/*shared_open_data_logistics_order*')
b2b_files = glob(f'{ed.raw_files}/*shared_open_data_b2b_order*')
b2c_files = glob(f'{ed.raw_files}/*nhm_order_fulfillment_subset_v1*')    
voucher_files = glob(f'{ed.raw_files}/*shared_open_data_gift_voucher_order*')

val_dict = {"logistics":logistics_files, 
            "retail_b2b":b2b_files, 
            "retail_b2c":b2c_files, 
            "voucher":voucher_files}

## Delete after this ...